In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from hmmlearn import hmm
import cvxpy as cp

# ==========================================
# 1. SETUP & DATA
# ==========================================
print("1. Initializing System...")
tickers = ['SPY', 'TLT', 'GLD'] 
raw_data = yf.download(tickers, start='2005-01-01', end='2024-01-01', auto_adjust=True)

# Handle yfinance columns
if 'Close' in raw_data.columns:
    data = raw_data['Close']
else:
    data = raw_data
data = data.sort_index(axis=1) # Sort alphabetically: GLD, SPY, TLT
actual_tickers = data.columns.tolist()
print(f"   Asset Order: {actual_tickers}")

returns = np.log(data / data.shift(1)).dropna()

# ==========================================
# 2. AI REGIME DETECTION
# ==========================================
print("2. Detecting Market Regimes...")
spy_col = 'SPY' if 'SPY' in data.columns else actual_tickers[1] 
spy_ret = returns[spy_col].values.reshape(-1, 1)
spy_vol = returns[spy_col].rolling(10).std().dropna().values.reshape(-1, 1)
spy_ret = spy_ret[9:] # Align lengths

X = np.column_stack([spy_ret, spy_vol])
model = hmm.GaussianHMM(n_components=3, covariance_type="full", n_iter=100, random_state=42)
model.fit(X)
hidden_states = model.predict(X)

# Map States to Logic
vol_by_state = pd.DataFrame(X, columns=['Ret', 'Vol']).groupby(hidden_states)['Vol'].mean()
sorted_idx = vol_by_state.sort_values().index
state_map = {sorted_idx[0]: 'BULL', sorted_idx[1]: 'CHOP', sorted_idx[2]: 'BEAR'}

# ==========================================
# 3. BACKTESTING ENGINE
# ==========================================
print("3. Running Backtest...")

def get_weights(mu, Sigma, mode):
    n = len(mu)
    w = cp.Variable(n)
    constraints = [cp.sum(w) == 1, w >= 0]
    
    if mode == 'BULL':
        risk = cp.quad_form(w, Sigma)
        ret = mu @ w
        obj = cp.Maximize(ret - 1.0 * risk)
    else: 
        risk = cp.quad_form(w, Sigma)
        obj = cp.Minimize(risk)
        
    try:
        cp.Problem(obj, constraints).solve(solver=cp.SCS, verbose=False)
        return w.value if w.value is not None else np.ones(n)/n
    except:
        return np.ones(n)/n

portfolio_values = [10000]
rebalance_freq = 20
backtest_returns = returns.iloc[9:] 
current_weights = np.ones(len(tickers)) / len(tickers)

for t in range(len(backtest_returns)):
    day_ret = backtest_returns.iloc[t].values
    port_ret = np.dot(current_weights, day_ret)
    portfolio_values.append(portfolio_values[-1] * (1 + port_ret))
    
    if t % rebalance_freq == 0 and t > 50:
        regime_id = hidden_states[t]
        market_mode = state_map[regime_id]
        past_data = backtest_returns.iloc[t-50:t] 
        mu = past_data.mean().values
        Sigma = past_data.cov().values
        current_weights = get_weights(mu, Sigma, market_mode)

# ==========================================
# 4. RESULTS
# ==========================================
print("4. Plotting Results...")
dates = backtest_returns.index
strategy_curve = pd.Series(portfolio_values[1:], index=dates)
spy_curve = (1 + backtest_returns[spy_col]).cumprod() * 10000

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), sharex=True)
ax1.plot(strategy_curve, label="Strategy", color='blue')
ax1.plot(spy_curve, label="SPY Benchmark", color='gray', linestyle='--')
ax1.legend()

colors = {sorted_idx[0]: 'green', sorted_idx[1]: 'yellow', sorted_idx[2]: 'red'}
# Plot every 5th day to speed up rendering
for i in range(0, len(dates)-1, 5): 
    s = hidden_states[i]
    ax2.axvspan(dates[i], dates[i+5], color=colors[s], alpha=0.3, linewidth=0)
    
plt.show()